<a href="https://colab.research.google.com/github/PSxUchiha/inpainting-gans/blob/main/fulfill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
! pip install -q kaggle


from google.colab import files

files.upload()

{}

In [47]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download 'paulchambaz/google-street-view'
! ls
! mkdir -p train/street
! unzip google-street-view.zip -d train/street

mkdir: cannot create directory ‘/root/.kaggle’: File exists
google-street-view.zip: Skipping, found more recently modified local copy (use --force to force download)
google-street-view.zip	kaggle.json  sample_data  train
Archive:  google-street-view.zip
replace train/street/dataset/0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [145]:
from __future__ import print_function
import argparse
import os
import random
import time
import math
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from PIL import Image
from torch.autograd import Variable


In [92]:
dataPath = 'train'
imageSize = 128
batchSize = 64
localSize = 64
niter = 25
preniter = 50
lr = 0.001
alpha = 0.01
beta1 = 0.5
beta2 = 0.99
hole_min = 32
hole_max = 48
workers = 2
cuda = 1
ndf = 64

In [93]:
manualSeed = random.randint(1,10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

print(f'CUDA device name: {torch.cuda.get_device_name("cuda:0")}')
device = torch.device("cuda:0" if cuda else "cpu")
print(device)
nc = 3

Random Seed:  8732
CUDA device name: Tesla T4
cuda:0


In [94]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [95]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(

            nn.Conv2d(nc, ndf, 5, stride=1, padding=2, dilation=1, bias=True),
            nn.BatchNorm2d(ndf),
            nn.ReLU(True),
            nn.Conv2d(ndf, ndf * 2, 2, stride=2, padding=0, dilation=1, bias=True),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            nn.Conv2d(ndf * 2, ndf * 2, 3, stride=1, padding=1, dilation=1, bias=True),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            nn.Conv2d(ndf * 2, ndf * 4, 2, stride=2, padding=0, dilation=1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 4, 3, stride=1, padding=1, dilation=1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 4, 3, stride=1, padding=1, dilation=1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 4, 3, stride=1, padding=2, dilation=2, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 4, 3, stride=1, padding=4, dilation=4, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 4, 3, stride=1, padding=8, dilation=8, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 4, 3, stride=1, padding=16, dilation=16, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 4, 3, stride=1, padding=1, dilation=1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 4, 3, stride=1, padding=1, dilation=1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ndf * 4, ndf * 2, 4, stride=2, padding=1, dilation=1, bias=True),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            nn.Conv2d(ndf * 2, ndf * 2, 3, stride=1, padding=1, dilation=1, bias=True),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ndf * 2, ndf, 4, stride=2, padding=1, dilation=1, bias=True),
            nn.BatchNorm2d(ndf),
            nn.ReLU(True),
            nn.Conv2d(ndf, int(ndf / 2), 3, stride=1, padding=1, dilation=1, bias=True),
            nn.BatchNorm2d(int(ndf / 2)),
            nn.ReLU(True),
            nn.Conv2d(int(ndf / 2), nc, 3, stride=1, padding=1, dilation=1, bias=True),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)

        return output


In [96]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.disc_d = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(ndf),
            nn.ReLU(True),
            nn.Conv2d(ndf, ndf * 2, 4, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),
            nn.Conv2d(ndf * 8, ndf * 8, 4, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),
            nn.Conv2d(ndf * 8, ndf * 8, 4, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True)
        )

        self.disc_l = nn.Sequential(
            nn.Conv2d(nc, ndf * 2, 4, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),
            nn.Conv2d(ndf * 8, ndf * 8, 4, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),
            nn.Conv2d(ndf * 8, ndf * 8, 4, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True)
        )

        self.classifier = nn.Sequential(
            nn.Conv2d(ndf * 16, 1, 1, stride=1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x1, x2):
        x_global = self.disc_d(x1)
        x_local = self.disc_l(x2)
        x = torch.cat((x_global, x_local), 1)
        output = self.classifier(x)

        return output.view(-1, 1).squeeze(1)

In [97]:
def weights_init(m):
    for m in m.modules():
        if isinstance(m, nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2./n))
            m.bias.data.zero_()
        elif isinstance(m, nn.ConvTranspose2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))
            m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.normal_(0, 1)
            m.bias.data.zero_()

In [98]:
def get_points():
    points = []
    mask = []
    for i in range(batchSize):
        x1, y1 = np.random.randint(0, imageSize - localSize + 1, 2)
        x2, y2 = np.array([x1, y1]) + localSize
        points.append([x1, y1, x2, y2])

        w, h = np.random.randint(hole_min, hole_max + 1, 2)
        p1 = x1 + np.random.randint(0, localSize - w)
        q1 = y1 + np.random.randint(0, localSize - h)
        p2 = p1 + w
        q2 = q1 + h

        m = np.zeros((1, imageSize, imageSize), dtype=np.uint8)
        m[:, q1:q2 + 1, p1:p2 + 1] = 1
        mask.append(m)

    return np.array(points), np.array(mask)

In [99]:
def crop_local_patches(images, points, bsize):
    cropped_images = torch.zeros([bsize, nc, imageSize // 2, imageSize // 2], dtype=torch.float32, device='cuda:0')
    for i in range(bsize):
        t = Variable(images[i])
        cropped_images[i] = t[:, points[i][0]:points[i][2], points[i][1]:points[i][3]]

    return cropped_images.cuda()

def save_checkpoint(state, curr_epoch):
    torch.save(state, './models/netG_e%d.pth.tar' % (curr_epoch))


### Initialising Generator and Discriminator

In [100]:
netG = Generator().to(device)
weights_init(netG)
print(netG)

netD = Discriminator().to(device)
weights_init(netD)
print(netD)

Generator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU(inplace=True)
    (15): Conv2d(256, 25

### Choosing Loss Function

In [101]:
criterion = nn.BCELoss()
criterion2 = nn.MSELoss()

### Initialise optimisers

In [102]:
real_label = 1
fake_label = 0
# setup optimizers
optG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))
optD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))

### Loading Data

In [103]:
traindir = os.path.join(dataPath, 'street')

train_loader = torch.utils.data.DataLoader(
        dsets.ImageFolder(traindir, transforms.Compose([
            transforms.Resize(128),
            #transforms.CenterCrop(128),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                std=[0.5, 0.5, 0.5])
        ])),
        batch_size=batchSize, shuffle=True,
        num_workers=workers, pin_memory=True)

errD_all = AverageMeter()
errG_all = AverageMeter()


### Training

In [108]:
for epoch in range(niter):
    t0 = time.time()
    for i, data in enumerate(train_loader, 0):

        # generate masks
        points_batch, mask_batch = get_points()
        real_data = data[0].to(device)
        batch_size = real_data.size(0)

        temp = torch.from_numpy(mask_batch)
        masks = temp.type(torch.FloatTensor).cuda()
        if real_data.size(0) < batchSize:
            masks_narrowed = masks.narrow(0, 0, real_data.size(0))
            masked_data = real_data * (1 - masks_narrowed)
        else:
            masked_data = real_data * (1 - masks)

        if epoch <= preniter:
            optG.zero_grad()
            gen_data = netG(masked_data)
            errG = criterion2(gen_data, real_data)
            errG.backward()
            optG.step()

            print('PRETRAIN [%d/%d][%d/%d] Loss_G: %.4f'
                  % (epoch, niter, i, len(train_loader), errG.item()))

            errG_all.update(errG.item())

        else:
            optD.zero_grad()
            local_real_data = crop_local_patches(real_data, points_batch, batch_size)
            label = torch.full((batch_size,), real_label, device=device)
            out = netD(real_data, local_real_data)
            errD_real = criterion(out, label)

            # train Discriminator with fake samples
            temp = torch.from_numpy(mask_batch)
            masks = temp.type(torch.FloatTensor).cuda()
            if real_data.size(0) < batchSize:
                masks_narrowed = masks.narrow(0, 0, real_data.size(0))
                masked_data = real_data * (1 - masks_narrowed)
            else:
                masked_data = real_data * (1 - masks)

            gen_data = netG(masked_data)
            label_f = label.clone()
            label_f.fill_(fake_label)
            local_gen_data = crop_local_patches(gen_data, points_batch, batch_size)
            out = netD(gen_data.detach(), local_gen_data.detach())
            errD_fake = criterion(out, label_f)
            errD = (errD_real + errD_fake) * alpha
            errD.backward()
            optD.step()

            # update Generator
            optG.zero_grad()
            out = netD(gen_data, local_gen_data)
            errG = criterion2(gen_data, real_data)
            errG.backward()
            optG.step()

            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f'
                % (epoch, niter, i, len(train_loader), errD.item(), errG.item()))

            errD_all.update(errD.item())
            errG_all.update(errG.item())



    print('Time elapsed Epoch %d: %d seconds'
          % (epoch, time.time() - t0))

    # save model parameters (last epoch)
    #save_checkpoint({
    #    'epoch': epoch + 1,
    #    'state_dict': netG.state_dict(),
    #    'optimizer': optG.state_dict(),
    #}, epoch+1)


PRETRAIN [0/25][0/157] Loss_G: 0.0257
PRETRAIN [0/25][1/157] Loss_G: 0.0235
PRETRAIN [0/25][2/157] Loss_G: 0.0214
PRETRAIN [0/25][3/157] Loss_G: 0.0210
PRETRAIN [0/25][4/157] Loss_G: 0.0219
PRETRAIN [0/25][5/157] Loss_G: 0.0207
PRETRAIN [0/25][6/157] Loss_G: 0.0206
PRETRAIN [0/25][7/157] Loss_G: 0.0196
PRETRAIN [0/25][8/157] Loss_G: 0.0222
PRETRAIN [0/25][9/157] Loss_G: 0.0211
PRETRAIN [0/25][10/157] Loss_G: 0.0228
PRETRAIN [0/25][11/157] Loss_G: 0.0173
PRETRAIN [0/25][12/157] Loss_G: 0.0262
PRETRAIN [0/25][13/157] Loss_G: 0.0195
PRETRAIN [0/25][14/157] Loss_G: 0.0195
PRETRAIN [0/25][15/157] Loss_G: 0.0179
PRETRAIN [0/25][16/157] Loss_G: 0.0167
PRETRAIN [0/25][17/157] Loss_G: 0.0186
PRETRAIN [0/25][18/157] Loss_G: 0.0186
PRETRAIN [0/25][19/157] Loss_G: 0.0194
PRETRAIN [0/25][20/157] Loss_G: 0.0188
PRETRAIN [0/25][21/157] Loss_G: 0.0230
PRETRAIN [0/25][22/157] Loss_G: 0.0276
PRETRAIN [0/25][23/157] Loss_G: 0.0230
PRETRAIN [0/25][24/157] Loss_G: 0.0201
PRETRAIN [0/25][25/157] Loss_G: 0.0

In [149]:
torch.save(netG, 'G_weights')
torch.save(netD, 'D_weights')


In [110]:
if not os.path.exists('outputs'):
  os.makedirs('outputs')


In [119]:
valdir = os.path.join(dataPath, 'val')

In [131]:
val_loader = torch.utils.data.DataLoader(
        dsets.ImageFolder(valdir, transforms.Compose([
            transforms.CenterCrop(128),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                std=[0.5, 0.5, 0.5])
        ])),
        batch_size=batchSize, shuffle=True,
        num_workers=workers, pin_memory=True)



In [158]:
# load the pretrained weights
valG = Generator().to(device)
state_dict = torch.load('G_weights')
print(state_dict)
#valG.load_state_dict(state_dict)
valG.eval()
#print(valG)



Generator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU(inplace=True)
    (15): Conv2d(256, 25

Generator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU(inplace=True)
    (15): Conv2d(256, 25

In [146]:
_errstr = "Mode is unknown or incompatible with input array shape."


def bytescale(data, cmin=None, cmax=None, high=255, low=0):
    if data.dtype == np.uint8:
        return data

    if high > 255:
        raise ValueError("`high` should be less than or equal to 255.")
    if low < 0:
        raise ValueError("`low` should be greater than or equal to 0.")
    if high < low:
        raise ValueError("`high` should be greater than or equal to `low`.")

    if cmin is None:
        cmin = data.min()
    if cmax is None:
        cmax = data.max()

    cscale = cmax - cmin
    if cscale < 0:
        raise ValueError("`cmax` should be larger than `cmin`.")
    elif cscale == 0:
        cscale = 1

    scale = float(high - low) / cscale
    bytedata = (data - cmin) * scale + low
    return (bytedata.clip(low, high) + 0.5).astype(np.uint8)


def toimage(arr, high=255, low=0, cmin=None, cmax=None, pal=None,
            mode=None, channel_axis=None):
    data = np.asarray(arr)
    if np.iscomplexobj(data):
        raise ValueError("Cannot convert a complex-valued array.")
    shape = list(data.shape)
    valid = len(shape) == 2 or ((len(shape) == 3) and
                                ((3 in shape) or (4 in shape)))
    if not valid:
        raise ValueError("'arr' does not have a suitable array shape for "
                         "any mode.")
    if len(shape) == 2:
        shape = (shape[1], shape[0])  # columns show up first
        if mode == 'F':
            data32 = data.astype(np.float32)
            image = Image.frombytes(mode, shape, data32.tostring())
            return image
        if mode in [None, 'L', 'P']:
            bytedata = bytescale(data, high=high, low=low,
                                 cmin=cmin, cmax=cmax)
            image = Image.frombytes('L', shape, bytedata.tostring())
            if pal is not None:
                image.putpalette(np.asarray(pal, dtype=np.uint8).tostring())
                # Becomes a mode='P' automagically.
            elif mode == 'P':  # default gray-scale
                pal = (np.arange(0, 256, 1, dtype=np.uint8)[:, np.newaxis] *
                       np.ones((3,), dtype=np.uint8)[np.newaxis, :])
                image.putpalette(np.asarray(pal, dtype=np.uint8).tostring())
            return image
        if mode == '1':  # high input gives threshold for 1
            bytedata = (data > high)
            image = Image.frombytes('1', shape, bytedata.tostring())
            return image
        if cmin is None:
            cmin = np.amin(np.ravel(data))
        if cmax is None:
            cmax = np.amax(np.ravel(data))
        data = (data*1.0 - cmin)*(high - low)/(cmax - cmin) + low
        if mode == 'I':
            data32 = data.astype(np.uint32)
            image = Image.frombytes(mode, shape, data32.tostring())
        else:
            raise ValueError(_errstr)
        return image

    if channel_axis is None:
        if (3 in shape):
            ca = np.flatnonzero(np.asarray(shape) == 3)[0]
        else:
            ca = np.flatnonzero(np.asarray(shape) == 4)
            if len(ca):
                ca = ca[0]
            else:
                raise ValueError("Could not find channel dimension.")
    else:
        ca = channel_axis

    numch = shape[ca]
    if numch not in [3, 4]:
        raise ValueError("Channel axis dimension is not valid.")

    bytedata = bytescale(data, high=high, low=low, cmin=cmin, cmax=cmax)
    if ca == 2:
        strdata = bytedata.tostring()
        shape = (shape[1], shape[0])
    elif ca == 1:
        strdata = np.transpose(bytedata, (0, 2, 1)).tostring()
        shape = (shape[2], shape[0])
    elif ca == 0:
        strdata = np.transpose(bytedata, (1, 2, 0)).tostring()
        shape = (shape[2], shape[1])
    if mode is None:
        if numch == 3:
            mode = 'RGB'
        else:
            mode = 'RGBA'

    if mode not in ['RGB', 'RGBA', 'YCbCr', 'CMYK']:
        raise ValueError(_errstr)

    if mode in ['RGB', 'YCbCr']:
        if numch != 3:
            raise ValueError("Invalid array shape for mode.")
    if mode in ['RGBA', 'CMYK']:
        if numch != 4:
            raise ValueError("Invalid array shape for mode.")

    # Here we know data and mode is correct
    image = Image.frombytes(mode, shape, strdata)
    return image


In [147]:

# sample a random batch and save 6 generated images for visualization
for i, data in enumerate(val_loader, 0):
    _, mask_batch = get_points()
    x = data[0].to(device)

    temp = torch.from_numpy(mask_batch)
    masks = temp.type(torch.FloatTensor).cuda()
    if x.size(0) < batchSize:
        masks_narrowed = masks.narrow(0, 0, x.size(0))
        masked_data = x * (1 - masks_narrowed)
    else:
        masked_data = x * (1 - masks)

    generated_samples = netG(masked_data)

    if (i % 10) == 0:
        out_path = './outputs/sample_' + str(i) + '.png'

        # save one of the generated images as example
        toimage(generated_samples.view(-1, 3, 128, 128)[0].cpu().detach().numpy()).save(out_path)
print(generated_samples[0][0])

tensor([[-0.0115, -0.0123, -0.0135,  ..., -0.0122, -0.0134, -0.0138],
        [-0.0063, -0.0132, -0.0123,  ..., -0.0134, -0.0132, -0.0178],
        [-0.0087, -0.0136, -0.0143,  ..., -0.0131, -0.0138, -0.0166],
        ...,
        [-0.0066, -0.0142, -0.0120,  ..., -0.0138, -0.0132, -0.0171],
        [-0.0101, -0.0156, -0.0156,  ..., -0.0150, -0.0148, -0.0173],
        [-0.0099, -0.0167, -0.0141,  ..., -0.0156, -0.0142, -0.0170]],
       device='cuda:0', grad_fn=<SelectBackward0>)


<ipython-input-146-cd9bf7f0465e>:102: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  strdata = np.transpose(bytedata, (1, 2, 0)).tostring()
